# Minimi quadrati ordinari e ricorsivi con NumPy

## Indice

1. [Diabetes dataset](#dataset)<br />
2. [Minimi quadrati ordinari](#ordinari)<br />
3. [Minimi quadrati ricorsivi](#ricorsivi)<br />
4. [Flusso di dati](#flusso)<br />

In [ ]:
import inspect
import matplotlib.pyplot as plt
import numpy as np

%load_ext autoreload
%autoreload 2

# 1. [Diabetes dataset](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset) <a id=dataset> </a>

In [ ]:
from sklearn import datasets

X, y = datasets.load_diabetes(return_X_y=True)
X = X[:, [2]] # tengo solo la variabile Body mass index

In [ ]:
from msbd.grafici import grafico_progressione_diabete_vs_bmi

print(inspect.getsource(grafico_progressione_diabete_vs_bmi))

grafico_progressione_diabete_vs_bmi(X, y)
plt.show()

# 2. Minimi quadrati ordinari <a id=ordinari> </a>

### Esercizio

1. Completare il metodo `fit()`, stima dei coefficienti con il metodo dei minimi quadrati ordinari, della classe `RegressioneLineare` in `msbd/modello_lineare/regressione_lineare.py`.

2. Verificare il proprio codice utilizzando pytest.

Si ricorda che la stima dei coefficienti $\hat{\beta}$ con il metodo dei minimi quadrati ordinari consiste in $\hat{\beta}=(X^TX)^{-1}X^Ty$.

In [ ]:
from msbd.modello_lineare import  RegressioneLineare

print(inspect.getsource(RegressioneLineare))

In [ ]:
!pytest -v msbd/tests/test_regressione_lineare_fit.py

In [ ]:
from sklearn.linear_model import LinearRegression

grafico_progressione_diabete_vs_bmi(X, y)

lr = LinearRegression()
lr.fit(X, y)
y_hat = lr.predict(X)

plt.plot(X, y_hat, c='tab:green', label="Minimi quadrati ordinari")
plt.legend()
plt.show()

### Esercizio

Riottenere il grafico utilizzando la classe `RegressioneLineare`.

# 3. Minimi quadrati ricorsivi <a id=ricorsivi> </a>

### Esercizio

1. Completare il metodo `partial_fit()`, stima dei coefficienti tramite l'algoritmo dei minimi quadrati ricorsivi, della classe `RegressioneLineare` in `msbd/modello_lineare/regressione_lineare.py`. 

2. Verificare il proprio codice utilizzando pytest.

Da pagina 34 del libro di B. Scarpa e A. Azzalini, *Data Analysis and Data Mining* (nota: la stima dell'errore standard è omessa):

**Algoritmo** Minimi quadrati ricorsivi
1. $W_{(p\times p)} \leftarrow 0, u_{(p\times 1)} \leftarrow 0.$
2. Ciclo per $n = 1, \dots, p$:
    1. leggo l'$n$-esimo record: $x \leftarrow \tilde{x}_n, y \leftarrow \tilde{y}_n$,
    2. $W \leftarrow W + xx^\top$,
    3. $u \leftarrow u + xy$.
    4. Se $n = p$:
        1. $V \leftarrow W^{-1}$.
        2. $\hat{\beta} \leftarrow Vu$.
3. Ciclo per $n = p + 1, p + 2, \dots$:
    1. leggo l'$n$-esimo record: $x \leftarrow \tilde{x}_n, y \leftarrow \tilde{y}_n$,
    2. $h = 1 / (1 + x^{\top}Vx)$,
    3. $e = y - x^{\top}\hat{\beta}$,
    4. $\hat{\beta} = \hat{\beta} + hVxe$,
    5. $V = V - hVxx^{\top}V$.

In [ ]:
!pytest -v msbd/tests/test_regressione_lineare_partial_fit.py

# 4. Flusso di dati <a id=flusso> </a>

In [ ]:
lr = LinearRegression()
lr.fit(X, y)
y_hat = lr.predict(X)

for i, (xn, yn) in enumerate(zip(X, y), 1):
    if i == 1:
        X_par = xn
        y_par = yn
    else:
        X_par = np.vstack([X_par, xn])
        y_par = np.vstack([y_par, yn])

    if i % 10 ** (1 + (i > 50)) == 0:
        lr = LinearRegression()
        lr.fit(X_par, y_par)
        y_hat_par = lr.predict(X)
    
        print("Regressione lineare dopo l'arrivo di {} osservazioni:".format(i))
        grafico_progressione_diabete_vs_bmi(X, y)
        plt.scatter(X[:i], y[:i], color="tab:orange")
        plt.plot(X, y_hat, color='tab:green', label="MQ ordinari - dataset completo")
        plt.plot(X, y_hat_par, color="tab:red", label="MQ ordinari - dataset parziale")
        plt.legend(loc="lower right")
        plt.show()

### Esercizio

1. Ripetere la simulazione del flusso di dati, questa volta confrontando la stima ottenuta con l'algoritmo dei minimi quadrati ordinari sul dataset completo con quella ottenuto con l'algoritmo dei minimi quadrati ricorsivi all'arrivo di nuove osservazioni. Riflettere attentamente a come deve essere modificato il codice utilizzato in precedenza.
2. Elencare i vantaggi di questo approccio rispetto al precedente.